In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Load data

Data coming from `vectorize.ipynb`, contains:

* `words`: Nx1 array of words to be considered
* `greens`: NxN matrix containing green matches of word $i$ with word $j$ for $(i,j) \in [0,N-1]^2$
* `yellows`: NxN matrix containing yellow matches of word $i$ with word $j$ for $(i,j) \in [0,N-1]^2$

`greens` and `yellows` are encoded as 5-bit numbers, with bit `0` corresponding to the first letter, bit `1` to the second, and so on.

So if $greens[i,j]$ = `20` = `0b10100` this means that when you type word $i$ and the secret word is word $j$, then the last and middle letters of $i$ will be green (read the code as little-endian correspondence with the string).

We`ll call $i$ the query index and $j$ the match index.

In [ ]:
read = np.load("matches.npz", allow_pickle=True)
greens = read["greens"]
yellows = read["yellows"]
words = pd.DataFrame(data=dict(words=read["words"]))

N = len(words)

print(f"{greens.shape=}")
print(f"{yellows.shape=}")
print(words)

# Entropy

To solve the tree we`ll need to calculate entropy. 

First, we join `greens` and `yellows` in a single matrix using a bitshift.

In [ ]:
M = greens.astype(np.uint16) + (yellows.astype(np.uint16) << 5)
del greens
del yellows
M

Now we calculate the item frequency in M.

(from this we could generate the plots from 3blue1brown`s video)

In [ ]:
def freqs(M):
    if M.ndim > 1:
        counts = [np.unique(m, return_counts=True)[1] for m in M]
        return [c/np.sum(c) for c in counts]
        # return [c for c in counts]
    else:
        counts = np.unique(M, return_counts=True)[1]
        s = np.sum(counts)
        return counts/s
            

information = [ np.sum(f*-np.log2(f)) for f in freqs(M) ]
infoDf = words.copy()
infoDf["information"] = information
infoDf

## The best word

The best first guess is the one that gives the most information:

In [ ]:
# word = words.iloc[np.argmax(information)].words
# print(f"{ np.max(information)=}")
# print(f"{word=}")
infoDf.sort_values(by="information", ascending=False).head(n=20)

In [ ]:
i = np.argmax(information)
f = freqs(M[i])
plt.bar(range(len(f)), sorted(f, key=lambda x: -x))

i = 25
f = freqs(M[i])
plt.bar(range(len(f)), sorted(f, key=lambda x: -x))

# Iteration!

You can solve Termo by iterating here, changing the variables "Guesses" and "Matches"

In [ ]:
guesses = ["cario","aruga","andem"]
matches = ["-yy--","gg--g","gy-y-"]

_bitValues = 2**np.arange(5)
def match_to_code(matchstring):
    green = 1*np.asarray([l=="g" for l in matchstring])
    yellow = 1*np.asarray([l=="y" for l in matchstring])
    return np.sum(_bitValues*green) + (np.sum(_bitValues*yellow) << 5)

subWords = words
subM = M
for guess, matchstr in zip(guesses,matches):
    match = match_to_code(matchstr)

    indexQuery = words.query("words==@guess").index[0]

    #(subM[indexQuery]!=0b0000011111)
    sub = np.where(subM[indexQuery]==match)[0]
    subM = subM[:,sub]

    subWords = subWords.iloc[sub]


information = [ np.sum(f*-np.log2(f)) for f in freqs(subM) ]
words["information"] = information
words["inSubset"] = 0
words.loc[subWords.index,"inSubset"] = 1
subWords.loc[:,"information"] = words.loc[subWords.index,"information"]
if(len(subWords) > 10):
    print(f"{ len(subWords)= }")
else:
    print(f"{len(subWords)} words left:")
    print(words.query("words in @subWords.words.values"))

print("Guesses:")
print(words.sort_values(by=[ "information","inSubset" ], ascending=False).head(n=10))

In [ ]:
subWords.sort_values(by="information", ascending=False)

In [ ]:
T = np.array([1,1,1.0])
f = freqs(T)
np.sum(f*-np.log2(f)) 

# Decision tree

In [ ]:
print("TODO!")